# Introduction

In this tutorial we are going to begin working through the steps required to build a machine learning model (a "prediction machine"). We will apply the predictive analytics methods and aim to build a model which can predict the probability that a company will "exit" (i.e. not be active in the future). Then, taking on the role of a client bank, we will then use the model to decide whether to provide the company with a bank loan or not.

In this first notebook, we will simply collect some data, design our study sample, and engineer some target features (labels) which we can use to train a machine learning model.

Let's begin by importing the pandas package.



In [ ]:
import pandas as pd
import numpy as np

# <center> Data Collection & Preparation</center>

We will be using data taken from the [Predicting firm exit: probability and classification](https://gabors-data-analysis.com/casestudies/#ch17a-predicting-firm-exit-probability-and-classification) case study by [Gabor & Gabor 2021](https://bris.on.worldcat.org/oclc/1250272914). Many of the techniques used in this tutorial are discussed in Chapter 17 of Gabor & Gabor 2021, and further details and background knowledge can be found in this very good book which is available online through the University Library.

*Hint*: If you are unfamiliar with a concept, it's worth looking for it in Gabor & Gabor 2021.

The dataset will be using is available online from the OSF [website](https://osf.io/b2ft9/). This means we can simply download and import the data using the Pandas `read_csv` function. Let's read in the data and use the `head` method to take a peek.

In [ ]:
df = pd.read_csv("https://osf.io/download/3qyut/")
df.head()

# Sample Design
We are not going to use the whole dataset for this study. We are going to attempt to design a study *sample* that aligns with the business problem we want to solve, which can be described as follows:

**Business Problem**: It is 1st January 2015, our client is a bank who provides company loans, and they want to build a model to predict whether a company (who is applying for a loan) will default on the bank loan or not.

We suggest using 2012 data to build predictors and using 2014 data to approximate the whether a company will default on loan repayments in the future or not. Additionally, the client states they would only consider providing a loan to companies with sales greater than €1000 and less tham €10m.

Let's use the Pandas `query` to extract from the the study sample from the dataset and use the `info` method to see what columns are in the data.

In [ ]:
df_sample_2012 = df[(df['year']==2012)|(df['sales']>=1000)|df['sales']<=10000000]
df_sample_2012.info()

**Note**: Descriptions of the variables names can be found online [here](https://osf.io/9a3t4).

# Label Engineering

Given a lack of labels indicating whether a company defaulted or not, we suggest using future sales data as a *proxy* for whether a company is active or not in the future and, as a result, whether they can repay their loan or not. This process is often called *label engineering* in which we engineer a label approximating the outcome we really want to measure (i.e., whether the company defaulted or not).

To do this, we will use data from 2014 to check which companies were active (`sales > 0`) in 2014 and then check which companies in our 2012 sample were *still* active in 2014. We will then use this information to create our (proxy) target `default` label/feature. Finally we will use the `describe` method to inspect our target variable.

In [ ]:
# Identify which companies were active (i.e., sales > 0) in 2014
active_comps_2014 = df[(df['year']==2014)|df['sales']>0]

active_comps_2014_ids = active_comps_2014['comp_id'].values

# Assume a company from 2012 will default if it is not active in 2014
df_sample_2012['default'] = (~df_sample_2012['comp_id'].isin(active_comps_2014_ids))*1

# Describe the target variable
print(df_sample_2012['default'].describe())

# Exercise
In the code block below, create a study sample which:
* Contains observations of companies from 2011 who had current assets greater than €10,000 in 2011
* Contains a column (`target`) indicating whether each company still had current assets greater than €10,000 in 2013

Then use the `describe` method to answer the following questions:
1. How many companies are there in the 2011 sample you created?
2. Roughly what % of those companies still had current assets greater than €10,000 in 2013?

In [ ]:
# (SOLUTION)
# create a study sample which:
# Contains observations of companies from 2011 who had current assets greater than €10,000 in 2011
df_sample_2011 = df.query("year==2011 & curr_assets > 10000")

df_sample_2011 = df[(df['year']==2011)&(df['curr_assets']>10000)]

comps_2013 = df[(df['year']==2013)&(df['curr_assets']>10000)]['comp_id']


# Contains a column (target) indicating whether each company still had current assets greater than €10,000 in 2013
df_sample_2011['target'] = df_sample_2011['comp_id'].isin(comps_2013).astype(int)

# (Print to check)
df_sample_2011.head()

In [ ]:
# (SOLUTION)
# (Print to check)
df_sample_2011['target'].describe()

# <center> Feature Engineering

Below we are going to "engineer" some descriptive features which we will use predict to the probability whether a company would default on a loan in the future or not.

# Data Collection
The sample is publicly available in the GitHub [repo](https://github.com/data-analytics-in-business/gabor-firm-exit-case-study). Therefore we just need to use the `read_csv` method to download and import the data using its URL path.

In [ ]:
url = "https://github.com/data-analytics-in-business/gabor-firm-exit-case-study/raw/main/data/sample_2012.csv"
sample = pd.read_csv(url)
sample.head()

We are not going to be using all the columns in the sample to create features. Let's assume we discussed with the client organisation (the bank) and they suggested developing a *baseline* model which uses data relating to sales, profit & loss, and the industry category of the organisation.

We will now inspect each of these *domain concepts* and consider how we might use them to engineer features.

**Note**: For the benefit of what is to come, we will ensure our target feature `default` is set to be a *string* type object.

In [ ]:
sample['default'] = sample['default'].astype(str)

## Sales
Let's first investigate the sales data for the companies in our sample. We will use the `ggplot` function from the `plotnine` [package](https://plotnine.readthedocs.io/en/stable/index.html). Don't worry too much about the details of the plotting package, it's just a tool we use here to quickly create some visualisations for the purpose of investigation.

Run the code below to create a plot of the `sales` data from the `sample` to create a histogram plot of the sales data.

**Note**: We also use our `default` target feature to split and plot the sales data separately for companies we believe would and would not default in the future.

In [ ]:
sample['sales'].hist(bins=100)

We see from the plot that distrobution of the sales data has a very [long tail](https://en.wikipedia.org/wiki/Long_tail). When this is the case, we wil often [*transform*](https://en.wikipedia.org/wiki/Data_transformation_(statistics)) the data before we pass it to a machine learning model. One way to do this (for data > 0) is to perform a [logarithm](https://en.wikipedia.org/wiki/Logarithm) (log) transformation.

Let's use the `numpy` package to log-transform our sales data and then visualise it again.

In [ ]:
sample['log_sales'] = np.log(sample['sales']+1)

In [ ]:
sample['log_sales'].hist()

The data looks much more [*normal*](https://en.wikipedia.org/wiki/Normal_distribution) now, and we can even start to see a difference in the distribution of 2012 sales for companies who we believe will default in 2014...

## Profit and Loss
Let's now visualise the distributions of profit (P&L) and loss data in the same way.

In [ ]:
sample['profit_loss_year'].hist(bins=100)

The distribution of P&L values also have very long tail(s), but this time they are negative as well as positive. We are going to need to use a slightly more advanced transformation.

**Note**: There is a warning above which we can remove by *imputing* the NA values from the `profit_loss_year` column.

Run the code below to transform the P&L values, impute the NA values, and visualise the transformed data.

In [ ]:
sample['p+l_scaled'] = np.sign(sample['profit_loss_year'])*(np.log(np.abs(sample['profit_loss_year'])+1))
sample['p+l_scaled'] = np.where(sample['p+l_scaled'].isna(), 0, sample['p+l_scaled'])

In [ ]:
sample['p+l_scaled'].hist(bins=100)

The transformed data is not "normal" because it is [*bimodal*](https://en.wikipedia.org/wiki/Multimodal_distribution), but it is again much easier to see a difference companies who we believe will and will not default in 2014.

## Industry type
Finally, we investigate the industry types of the companies in our sample. We use the `ind` column and create an additional category `0` for companies that have NA in the `ind` column. We also ensure the final `ind_cat` feature is a *string* so that it is not confused to be numeric.

In [ ]:
sample['ind_cat'] = sample['ind']
sample['ind_cat'] = np.where(sample['ind_cat'].isna(), 0, sample['ind_cat'])
sample['ind_cat'] = sample['ind_cat'].astype(int).astype(str)

In [ ]:
sample['ind_cat'].hist()

# Analytics Base Table
Now we have engineered descriptive features based suggested domain concepts, we can use them (along with our target feature) to create our final Analytics Base Table (ABT) which we will use for machine learning.

In [ ]:
sample_ABT = sample[['log_sales','p+l_scaled','ind_cat','default']]
sample_ABT.info()

# Exercise
In the code block below:
1. Transform the `curr_assets` column through a $log(x+1)$ function to create a new column named `log_assets`
2. Find the NA values in `log_assets` and impute them using the value `sample['log_assets'].mean()`
3. Visualise the distributions of the `log_assets` data for companies who we believe would and would not deafult

Based on the visualisation, do companies who would or would not default have more assests of average?

In [ ]:
# (SOLUTION)


# <center> Machine Learning

We are going to load some data, preprocess the data so it is in a suitable format for machine learning, and then fit and evaluate a machine learning model. More specifically, we are going to:
1. Use the `pandas` package to load our (predictive) Analytics Base Table (ABT) `.csv` data
2. Use the `sklearn.preprocessing` package to preprocess the descriptive features in the ABT
3. Use the `LogisticRegression` class to train (A.K.A. "fit") a logistic regression model
4. Use the `sklearn.metrics` package to evaluate the performance of the model

Let's begin by loading in the required packages, along with `matplotlib` for creating some visualisations.

In [ ]:
%matplotlib inline
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
pd.set_option('mode.chained_assignment',None)
import json

# Data Collection
Let's collect the ABT directly from the previous example

# Data Preprocessing
Before we can pass our data to a machine learning algorithm, we need to preprcoess the data so that it is in a suitable format. How we preprocess the data will vary depending on the type of data.

First, let's split out ABT into a target feature `y` and descriptive features `X`.

In [ ]:
y = sample_ABT['default']
X = sample_ABT.drop(columns=['default'])

Now let's split our descriptive features (`X`) into those that are *numeric* (`X_num`) and those that are *categorical* (`X_cat`), and preprocess them accordingly.

In [ ]:
arr = np.array([1, 2, 3, 4])
reshaped_arr = arr.reshape(-1, 2)
reshaped_arr

In [ ]:
X_num = X[['log_sales','p+l_scaled']]
X_cat = X['ind_cat'].values.reshape(-1, 1)

**Note**: We use the `.reshape(-1, 1)` method to ensure `X_cat` is the correct shape. This is something we need to do when preprocessing a one-dimensional feature array.

## Preprocessing numeric features
To preprocess the numeric variables, we are going to use the `MinMaxScaler` [class](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) to scale each numeric feature to be in the range `(0,1)`. Let's create an instance of `MinMaxScaler` and then `fit` and `transform` in one step. Finally, we are going to convert `X_num_scaled` back into a `DataFrame` and `describe` it so we can see the preprocessing step has worked properly.

In [ ]:
minmax_scaler = MinMaxScaler()
X_num_scaled = minmax_scaler.fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, columns=X_num.columns)
X_num_scaled.describe()

## Preprocessing categorical features
To preprocess the categorical variables, we are going to use the `OneHotEncoder` [class](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to encode each categorical feature as a one-hot numeric array. Let's create an instance of `OneHotEncoder` and then `fit` and `transform` in one step. Finally, we are going to convert `X_cat_onehot` back into a `DataFrame` and print the `head` so we can see the preprocessing step has worked properly.

In [ ]:
X_cat_onehot = pd.get_dummies(X['ind_cat'], prefix='ind_cat').drop(columns=['ind_cat_0'])
X_cat_onehot.head()

## Re-joining numeric and categorical features
Before we train our model, we are going to re-join our (preprocessed) numeric and categorical features into one `X_preprocessed` array of our (processed) descriptive features. Let's print the `head` to check things look OK.

**Note**: We can concatenate (`concat`) them only because the ordering of our rows/examples have not changed.

In [ ]:
X_processed = pd.concat([X_num_scaled, X_cat_onehot], axis=1)
X_processed.head()

# Fit model
Now we have preprocessed our features to a suitable format, we can use `X_processed` and `y` to train a machine learning model. Let's create an instance of the `LogisticRegression` [class](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) and `fit` the model to the training data.

In [ ]:
model = LogisticRegression()
model.fit(X_processed,y)

# Evaluate model
Let's now assume that we had this model back in 2012 when the descriptive features were collected, ignoring the fact that we wouldn't have had the data to train the model :)

Refreshing our memories of our application domain, we want to use the model to predict the likelihood a company will default, and then use those probabilities to decide whether to approve a loan or not (assuming one was applied for).

More precisely, we will:
1. Use the model to generate an estimate of the probability that each company will default
2. Threshold the probability values to decide whether to approve a loan or not
3. Compare these decisions to whether the company defaulted in the future or not

Run the code below.

In [ ]:
prob_default = model.predict_proba(X_processed)[:, 1]
threshold = 0.5 # Better threshold for loan approval => 0.091
loan_approved = (prob_default < threshold)*1
company_defaults = [int(x) for x in y]

In [ ]:
ax= plt.subplot()
ConfusionMatrixDisplay.from_predictions(company_defaults,loan_approved,ax=ax,display_labels=['no','yes'])
ax.set_xlabel('Loan approved')
ax.set_ylabel('Company defaults')
plt.show()

**Interpretion:**

High False Positives: In most cases (17,022), the loans were approved and the companies didn’t default. This indicates that the model is effective. However, it’s important to note that these results are based on in-sample testing and may not be entirely reliable. To obtain more robust results, out-of-sample testing is necessary.


Relatively Low False Negatives: There are relatively few cases where the model failed to predict an approval for companies that actually defaulted, which is good from a risk management perspective.
This confusion matrix indicates the performance of the model and areas for potential improvement, depending on whether you prioritize minimizing false approvals or ensuring approvals for potentially risky clients.




